In [ ]:
import os
import errno
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pylab
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
ndf=32
ngf=32
nc=3
#final image size
fs=ndf*2//32

<h1> Load pre-trained model </h1>

In [ ]:
channels = 3
leak = 0.1
w_g = 4
def _make_dir(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

class GenerativeNet(torch.nn.Module):
    
    def __init__(self,z_dim):
        super(GenerativeNet,self).__init__()
        
        self.z_dim = z_dim

        self.model = nn.Sequential(
            nn.ConvTranspose2d(z_dim, 512, 4, stride=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=(1,1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=(1,1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=(1,1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, channels, 3, stride=1, padding=(1,1)),
            nn.Tanh())

    def forward(self, z):
        return self.model(z.view(-1, self.z_dim, 1, 1))

In [ ]:
# FID to compute
#1 "SN_Rg-DCGAN-Evidential-(2-2)-Conv"
num_gpu = 1 if torch.cuda.is_available() else 0
G = GenerativeNet(100)
dataset='CIFAR10'
model=["EviD-GAN-Continuous"] # 

<h1> Generate some images </h1>

In [ ]:
for l in model:
    for k in range(96): #146 251 296; 63-96
        # load weights
        G.load_state_dict(torch.load("D:\data\models\{}\{}\G_epoch_{}".format(l,dataset,((k*2000)+10000))))
        G.eval()
        with torch.no_grad():
            if torch.cuda.is_available():
                #D = D.cuda()
                G = G.cuda()
            batch_size = 100
            latent_size = 100
            # Directory for generated images
            outDir="D:\g_sample\{}\{}\G_epoch_{}\\1\\".format(l,dataset,(k*2000)+10000)
            _make_dir(outDir)
            for i in range(125):
                fixed_noise = torch.randn(batch_size, latent_size)
                if torch.cuda.is_available():
                    fixed_noise = fixed_noise.cuda()
                fake_images = G(fixed_noise)
                for j in range(batch_size):
                    torchvision.utils.save_image(fake_images[j], outDir+str(100*i+j)+".png", padding=0, normalize=True, scale_each=True, pad_value=0)
torch.cuda.empty_cache()
os._exit(00)